In [30]:
import pandas as pd
import numpy as np

# 导入数据，路径中要么用\\或/或者在路径前加r
dataset = pd.read_csv(r'Label_synergy4000.csv').set_index(["Unnamed: 0"])

# 输出数据预览
dataset

,A2058_SKIN,A2780_OVARY,A375_SKIN,A427_LUNG,CAOV3_OVARY,COLO320_LARGE_INTESTINE,EFM192A_BREAST,ES2_OVARY,HCT116_LARGE_INTESTINE,HT144_SKIN,...,SKMES1_LUNG,SKOV3_OVARY,SW620_LARGE_INTESTINE,SW837_LARGE_INTESTINE,T47D_BREAST,UACC62_SKIN,VCAP_PROSTATE,ZR751_BREAST,DLD1_LARGE_INTESTINE,PA1_OVARY
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,-9.51,2.60,15.20,6.22,-16.4000,5.360,-7.080,8.050,12.400,5.79,...,-6.230,13.50,5.10,5.15,6.63,3.880,-1.48,-8.660,13.00,4.83
1,-13.20,-4.03,11.10,10.50,-15.6000,16.300,-1.720,8.890,0.686,9.68,...,2.820,1.51,13.80,-5.77,-14.40,-10.700,-16.80,-11.800,15.30,-13.10
2,26.40,14.50,29.50,17.50,17.7000,10.300,0.830,5.770,19.300,8.40,...,-12.200,-7.40,10.50,-4.97,13.10,18.800,12.00,5.950,6.77,18.90
3,4.33,-8.16,-5.42,-7.75,-14.3000,4.810,0.312,3.410,2.920,-3.48,...,-3.140,-13.90,-4.57,-19.60,-2.72,3.680,14.40,-13.600,-3.64,-5.73
4,45.50,22.50,22.80,18.00,20.1000,22.700,6.300,24.900,35.400,23.40,...,24.400,31.20,18.40,12.90,14.30,34.700,21.20,14.400,26.40,55.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,1.35,11.70,20.20,9.17,46.6000,0.586,-37.700,-2.100,34.100,18.30,...,-0.217,-10.90,22.00,-2.43,-43.90,9.340,-4.83,-15.400,-4.42,17.80
579,9.19,8.01,7.50,-4.18,-6.7800,3.210,7.240,5.340,9.690,4.67,...,11.100,7.12,-2.42,6.96,14.90,9.120,10.60,14.700,11.60,14.20
580,15.40,12.40,19.20,-5.01,37.4000,8.590,9.770,3.770,10.600,18.20,...,4.330,5.84,-1.58,18.60,-20.70,25.100,20.30,3.750,-4.29,-1.44


In [31]:
columns = dataset.columns
del dataset[columns[0]]

In [32]:
dict_xy = {}
for column in dataset.columns:
    dict_xy[column] = {}
    temp = dataset.copy()
    del temp[column]
    dict_xy[column]["X"] = temp.values
    dict_xy[column]["y"] = dataset[column].values

In [33]:
temp_mean = 0
for elem in dict_xy:
    # 准备训练数据
    # 自变量：汽油税、人均收入、高速公路、人口所占比例
    # 因变量：汽油消耗量
    # X = dataset.iloc[:, 1:34].values
    # y = dataset.iloc[:, 35].values
    X = dict_xy[elem]["X"]
    y = dict_xy[elem]["y"]

    # 将数据分为训练集和测试集
    from sklearn.model_selection import train_test_split

    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        test_size=0.8,
                                                        random_state=0)

    # 特征缩放，通常没必要
    # 因为数据单位，自变量数值范围差距巨大，不缩放也没问题
    from sklearn.preprocessing import StandardScaler

    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)

    # 训练随机森林解决回归问题
    from sklearn.ensemble import RandomForestRegressor

    regressor = RandomForestRegressor(n_estimators=200, random_state=0)
    regressor.fit(X_train, y_train)
    y_pred = regressor.predict(X)

    # 评估回归性能
    from sklearn import metrics
    print(elem)
    print('Mean Absolute Error:', metrics.mean_absolute_error(y, y_pred))
    print('Mean Squared Error:', metrics.mean_squared_error(y, y_pred))
    temp_mean += metrics.mean_squared_error(y, y_pred)
    print('Root Mean Squared Error:',
          np.sqrt(metrics.mean_squared_error(y, y_pred)))
    print()
mean_mean_squared_error = temp_mean/len(dataset.columns)
mean_mean_squared_error

A2780_OVARY 
Mean Absolute Error: 33.34631987993139
Mean Squared Error: 1403.7687719332332
Root Mean Squared Error: 37.46690235305333

A375_SKIN 
Mean Absolute Error: 21.061106946826747
Mean Squared Error: 714.0022305027794
Root Mean Squared Error: 26.720820168976463

A427_LUNG 
Mean Absolute Error: 20.506375958833626
Mean Squared Error: 628.7461859449597
Root Mean Squared Error: 25.074811782842154

CAOV3_OVARY 
Mean Absolute Error: 46.98511601200684
Mean Squared Error: 2851.743802311815
Root Mean Squared Error: 53.40172096769743

COLO320_LARGE_INTESTINE 
Mean Absolute Error: 18.716562551457955
Mean Squared Error: 491.51196733127824
Root Mean Squared Error: 22.17006917741301

EFM192A_BREAST 
Mean Absolute Error: 22.767318722126923
Mean Squared Error: 752.5646638162186
Root Mean Squared Error: 27.43291205497912

ES2_OVARY 
Mean Absolute Error: 20.255632180102943
Mean Squared Error: 631.1680452660977
Root Mean Squared Error: 25.123058039699263

HCT116_LARGE_INTESTINE 
Mean Absolute Error

1251.332549270158

In [34]:
from sklearn.linear_model import LinearRegression
X_train, X_test, y_train, y_test = train_test_split(
      X, y, test_size=0.2,random_state=1)
lr = LinearRegression()
lr.fit(X_train, y_train)
print('Slope: ',lr.coef_)
print('Intercept: ',lr.intercept_)

from sklearn import metrics
print("Explained Variance Score: ", 
    metrics.explained_variance_score(y_test, y_pred))
print('Mean Absolute Error:', 
    metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', 
    metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', 
    np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Slope:  [ 0.1756365   0.06166263  0.087274    0.01467673  0.01547289 -0.03297203
 -0.131742    0.15934526  0.04010011  0.23388174  0.00083216  0.12552173
 -0.00255466 -0.02804203 -0.19847455 -0.01487808  0.05295053  0.12175094
  0.07587161 -0.09746418  0.07080406 -0.02258201  0.2372599  -0.00390535
  0.03815272  0.18821507 -0.04206567 -0.06717769  0.02346031  0.0449994
 -0.06357087 -0.12013486  0.03324103]
Intercept:  -2.203298506260035


ValueError: Found input variables with inconsistent numbers of samples: [117, 583]